# Stream Analysis Flow
This notebook processes YouTube transcripts and live chat logs to analyze engagement, spikes, and keywords.

## 1. Workspace Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import re
from collections import Counter
import nltk
from nltk.corpus import stopwords
import textwrap

# Custom module
import parsers
import youtube_client # Ensure this file exists in your directory

# Pre-download NLTK data
nltk.download("stopwords", quiet=True)
stops = set(stopwords.words("english"))

# Chart styling
plt.style.use('seaborn-v0_8-darkgrid')

In [ ]:
# --- HELPER FUNCTIONS ---
def extract_transcript_by_minutes(transcript_df, minutes):
    """
    Retrieves subtitle text for specific minutes.
    
    Args:
        transcript_df (pd.DataFrame): The dataframe containing transcript data.
        minutes (list or int): A single minute (int) or list of minutes to retrieve.
        
    Returns:
        dict: A dictionary where key is the minute (int) and value is the text (str).
    """
    if isinstance(minutes, (int, float)):
        minutes = [int(minutes)]
        
    results = {}
    for m in minutes:
        start_sec = m * 60
        end_sec = (m + 1) * 60
        
        # Filter rows that start within this minute
        mask = (transcript_df['offset_start_seconds'] >= start_sec) & \
               (transcript_df['offset_start_seconds'] < end_sec)
               
        segment = transcript_df[mask]
        if not segment.empty:
            # Join text and clean up extra spaces
            text = " ".join(segment['text'].tolist())
            results[m] = text.strip()
        else:
            results[m] = "(No speech detected)"
            
    return results

In [ ]:
# Configuration
YOUTUBE_URL = "https://www.youtube.com/watch?v=XKGVM86HLOQ"
YT_ID = YOUTUBE_URL.split("=")[-1].split("?")[0]

print(f"Targeting Video ID: {YT_ID}")

## 2. Download & Parse Data

In [ ]:
# Download source materials
transcript_filepath = youtube_client.download_transcript(YT_ID)
chat_filepath = youtube_client.download_live_chat(YT_ID)

# Parse DataFrames
parsed_transcript_df = pd.DataFrame()
parsed_chat_df = pd.DataFrame()

if transcript_filepath:
    parsed_transcript_df = parsers.parse_transcript_vtt(transcript_filepath)
else:
    print(f"No transcript available for: {YT_ID}")

if chat_filepath:
    parsed_chat_df = parsers.parse_live_chat_json(chat_filepath)
else:
    print(f"No live chat available for: {YT_ID}")

# Display Info
print("\n--- Transcript Info ---")
parsed_transcript_df.info()
print("\n--- Chat Info ---")
parsed_chat_df.info()

## 3. General Activity Analysis
Identify moments where chat volume spikes significantly.

In [ ]:
if not parsed_chat_df.empty:
    # Aggregate by minute
    messages_per_minute = parsed_chat_df.groupby("minute").size().rename("message_count").reset_index()
    
    # Peak Detection
    counts = messages_per_minute["message_count"].values
    # Peak threshold: Mean + 2 Standard Deviations
    threshold = counts.mean() + (2 * counts.std())
    peaks, _ = find_peaks(counts, height=threshold)
    peak_minutes = messages_per_minute.loc[peaks, "minute"].tolist()

    # Plotting
    plt.figure(figsize=(15, 6))
    bars = plt.bar(messages_per_minute['minute'], messages_per_minute['message_count'], 
                   color='skyblue', label='Messages/Min')

    # Highlight spikes
    for minute in peak_minutes:
        idx = messages_per_minute[messages_per_minute['minute'] == minute].index
        if not idx.empty:
            bars[idx[0]].set_color('salmon')

    plt.title(f'Chat Volume per Minute (Spikes > {int(threshold)} msgs)', fontsize=14)
    plt.xlabel('Minute Offset')
    plt.ylabel('Message Count')
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    # Show context for peaks using the new helper function
    if not parsed_transcript_df.empty and peak_minutes:
        print(f"\n--- CONTEXT FOR HIGH TRAFFIC MOMENTS (>{int(threshold)} msgs) ---")
        transcript_context = extract_transcript_by_minutes(parsed_transcript_df, peak_minutes)
        
        for m in sorted(peak_minutes):
            print(f"\n[Minute {m}] {messages_per_minute.loc[messages_per_minute['minute'] == m, 'message_count'].values[0]} msgs")
            print(f"Streamer said: {textwrap.fill(transcript_context[m], width=80, initial_indent='  ', subsequent_indent='  ')}")
else:
    print("Skipping analysis: No chat data.")

## 4. User Analysis

In [ ]:
if not parsed_chat_df.empty:
    # Calculate Counts
    user_counts = parsed_chat_df["author_name"].value_counts().reset_index()
    user_counts.columns = ["author_name", "msg_count"]

    # --- Gini & Lorenz Curve ---
    counts = user_counts["msg_count"].values
    counts_sorted = np.sort(counts)
    n = len(counts_sorted)
    
    # Lorenz calc
    cum_counts = np.cumsum(counts_sorted)
    normalized_cum_counts = cum_counts / cum_counts[-1]
    
    # Gini calc
    # Area under Lorenz curve is sum(cum_counts) / total_sum / n roughly, 
    # but standard formula is (2 * Area_between_line_equality_and_lorenz)
    gini = (n + 1 - 2 * np.sum(cum_counts) / cum_counts[-1]) / n

    x_axis = np.linspace(0, 1, len(normalized_cum_counts))

    plt.figure(figsize=(6, 6))
    plt.plot(x_axis, normalized_cum_counts, label=f'Gini: {gini:.3f}', linewidth=2)
    plt.plot([0, 1], [0, 1], 'k--', alpha=0.5, label='Equality')
    plt.title('Lorenz Curve (Chat Inequality)')
    plt.xlabel('Cumulative % of Users')
    plt.ylabel('Cumulative % of Messages')
    plt.legend()
    plt.show()

    print(f"Top 5 Chatters:\n{user_counts.head(5)}")

## 5. Keyword & Transcript Deep Dive (New Feature)
Define a list of keywords to see where they appear in chat, and cross-reference with what was being said in the video.

In [ ]:
# --- USER CONFIG ---
TARGET_KEYWORDS = ["lmao", "lol", "wow", "gg", "kekw"] 
MIN_HIT_THRESHOLD = 5 # Only show subtitles if keywords appear > this many times in a minute
# ------------------

if not parsed_chat_df.empty and not parsed_transcript_df.empty:
    
    # 1. Filter chat for keywords
    keyword_hits = []
    
    print("Scanning chat for keywords...")
    for idx, row in parsed_chat_df.iterrows():
        msg_lower = str(row['message']).lower()
        minute = row['minute']
        
        for kw in TARGET_KEYWORDS:
            if kw in msg_lower:
                keyword_hits.append({
                    'minute': minute,
                    'keyword': kw
                })
    
    df_hits = pd.DataFrame(keyword_hits)
    
    if not df_hits.empty:
        # 2. Pivot for Stacked Bar Chart
        pivot_df = df_hits.groupby(['minute', 'keyword']).size().unstack(fill_value=0)
        
        all_minutes = range(int(parsed_chat_df['minute'].min()), int(parsed_chat_df['minute'].max()) + 1)
        pivot_df = pivot_df.reindex(all_minutes, fill_value=0)

        # 3. Plot
        ax = pivot_df.plot(kind='bar', stacked=True, figsize=(18, 8), width=1.0, colormap='viridis')
        
        ticks = ax.xaxis.get_ticklocs()
        ticklabels = [l.get_text() for l in ax.xaxis.get_ticklabels()]
        ax.xaxis.set_ticks(ticks[::10])
        ax.xaxis.set_ticklabels(ticklabels[::10], rotation=0)
        
        plt.title(f'Keyword Frequency per Minute: {TARGET_KEYWORDS}', fontsize=16)
        plt.xlabel('Minute Offset')
        plt.ylabel('Frequency')
        plt.legend(title='Keywords')
        plt.show()
        
        # 4. Context Extraction (Subtitles)
        # Calculate total hits per minute
        total_hits_series = pivot_df.sum(axis=1)
        
        # Filter: Keep only minutes >= threshold
        significant_minutes = total_hits_series[total_hits_series >= MIN_HIT_THRESHOLD]
        
        # Sort the significant ones by activity
        top_minutes = significant_minutes.sort_values(ascending=False).head(3).index.tolist()
        
        if top_minutes:
            print(f"\n--- TRANSCRIPT CONTEXT FOR TOP SPIKES (Threshold: {MIN_HIT_THRESHOLD}+ hits) ---")
            
            # --- USE NEW HELPER FUNCTION HERE ---
            transcript_context = extract_transcript_by_minutes(parsed_transcript_df, top_minutes)
            
            for m in sorted(top_minutes):
                hits_in_minute = total_hits_series.loc[m]
                breakdown = dict(pivot_df.loc[m][pivot_df.loc[m] > 0])
                print(f"\n[Minute {m}] Total Hits: {hits_in_minute} | Keywords: {breakdown}")
                print(f"{textwrap.fill(transcript_context[m], width=80, initial_indent='  ', subsequent_indent='  ')}")
        else:
            print(f"\nNo minutes found with more than {MIN_HIT_THRESHOLD} keyword hits. Try lowering the threshold.")
                
    else:
        print("No matches found for the provided keywords.")
else:
    print("Chat or Transcript data missing, cannot run Deep Dive.")